In [1]:
# Allows code to live reload
%load_ext autoreload
%autoreload 2

## 1. Install
```sh
# Install prerequisites
pip install pyodc

```

## 2. Make an ECMWF account
- Go to ecmwf.int/, click login at the top right and click register to make a new account.
- Once logged in, go to api.ecmwf.int/v1/key/ to get your key. 
- Put it in `~/.ecmwfapirc` as directed.

In [2]:
# Load in the ECMWF token 
from pathlib import Path
import json
import requests
from IPython.display import JSON
from datetime import datetime as dt
from datetime import timedelta, timezone
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

with open(Path("~/.ecmwfapirc").expanduser(), "r") as f:
    api_creds = json.load(f)

print("Checking API credentials")
r = requests.get(f"https://api.ecmwf.int/v1/who-am-i?token={api_creds['key']}")
if r.status_code == 403: print("Your credentials are either wrong or need to be renewed at https://api.ecmwf.int/v1/key/")
r.raise_for_status()
JSON(r.json())

Checking API credentials


<IPython.core.display.JSON object>

In [3]:
session = requests.Session()
session.headers["Authorization"] = f"Bearer {api_creds['key']}"

In [4]:
url = "https://ionbeam-dev.ecmwf.int/api/v1/"

In [5]:
stations = session.get(url + "stations").json()
print(f"{len(stations) = }")
# print(json.dumps(stations[0], indent = 4))

len(stations) = 3853


In [7]:
from collections import Counter
Counter(s["platform"] for s in stations)

Counter({'meteotracker': 3853})

## Requesting all the observed parameters from a station

Each station is associated with one or more sensor objects which each measure one or more observed property.

In [6]:
from datetime import datetime

# Obtain a station of interest
station = stations[-1]

for sensor in station["sensors"]:
    for property in sensor["properties"]:
        print(property)

{'key': 'author', 'name': 'author', 'unit': None, 'description': 'The author field from the Meteotracker data', 'url': None}
{'key': 'track_id', 'name': 'track_id', 'unit': None, 'description': 'A unique identifer for a single recorded track from a mobile sensor.', 'url': None}
{'key': 'dew_point_temperature', 'name': 'dew_point_temperature', 'unit': 'K', 'description': None, 'url': None}
{'key': 'air_temperature_near_surface', 'name': 'air_temperature_near_surface', 'unit': 'K', 'description': None, 'url': None}
{'key': 'solar_radiation_index', 'name': 'solar_radiation_index', 'unit': '1', 'description': None, 'url': None}
{'key': 'relative_humidity_near_surface', 'name': 'relative_humidity_near_surface', 'unit': '%', 'description': None, 'url': None}
{'key': 'time', 'name': 'time', 'unit': None, 'description': 'The time that the observation was made.', 'url': None}
{'key': 'humidity_index', 'name': 'humidity_index', 'unit': 'K', 'description': None, 'url': None}
{'key': 'offset_tz', 

In [7]:
# Polytope can also be used as a pure REST API, instructions here: https://polytope-client.readthedocs.io/en/latest/client/rest_api.html
from polytope.api import Client
import pandas as pd
from io import BytesIO

polytope_client = Client(address='polytope-test.ecmwf.int', 
                verbose=False, insecure=False, 
                user_email = api_creds["email"], 
                user_key = api_creds["key"])

### Obtain a single data granule in its entirety

In [8]:
# Obtain a particular station by id


station = max(stations, key = lambda s : dt.fromisoformat(s["stop_time"]))

source_id = station["external_id"]
platform = station["platform"]
parameter = "air_temperature_near_surface"

start_time = datetime.fromisoformat(station["start_time"])
stop_time = datetime.fromisoformat(station["stop_time"])
timespan = stop_time - start_time

print(f"""
{source_id = }
{platform = }
{parameter = }
{start_time = }
{stop_time = }
{timespan = }
""")

mars_request = {
    'project' : 'public',
    'platform' : 'meteotracker',
    'variable' : parameter,
    'datetime' : start_time,
    'format' : 'csv' # json, csv or odb
}

data = polytope_client.retrieve('iot', mars_request)
print(f"{len(data) = }")
df = pd.read_csv(BytesIO(data[0]))

for column in df.columns:
    if df[column].nunique() < 10: print(f"{column} unique entries: {df[column].unique()}")

df

2024-10-09 13:58:57 - INFO - Sending request...
{'request': 'datetime: 2024-04-09 11:31:24.011000+00:00\n'
            'format: csv\n'
            'platform: meteotracker\n'
            'project: public\n'
            'variable: air_temperature_near_surface\n',
 'verb': 'retrieve'}



source_id = '66152710f5c2613269425a01'
platform = 'meteotracker'
parameter = 'air_temperature_near_surface'
start_time = datetime.datetime(2024, 4, 9, 11, 31, 24, 11000, tzinfo=datetime.timezone.utc)
stop_time = datetime.datetime(2024, 4, 9, 11, 33, 20, 11000, tzinfo=datetime.timezone.utc)
timespan = datetime.timedelta(seconds=116)



2024-10-09 13:58:58 - INFO - Request accepted. Please poll ../requests/783ee4d5-d010-44d5-a816-d6e59ffa0453 for status
2024-10-09 13:58:58 - INFO - Checking request status (783ee4d5-d010-44d5-a816-d6e59ffa0453)...
2024-10-09 13:58:59 - INFO - The current status of the request is 'queued'
2024-10-09 13:59:00 - INFO - The current status of the request is 'processing'
2024-10-09 13:59:01 - INFO - The current status of the request is 'processed'
2024-10-09 13:59:01 - INFO - Starting data download (application/octet-stream)...


len(data) = 1
platform unique entries: ['meteotracker']
source_name unique entries: ['genova_living_lab_1' 'bologna_living_lab_3' 'bologna_living_lab_13'
 'bologna_living_lab_4']
source_id unique entries: ['62ab72c11d8e11061d32002a' '62b47f70c93e4f0780cf68da'
 '62b5dfeb82a7680332841b58' '62bda5d4edb2ca4efd101c4d']
observation_variable unique entries: ['air_temperature_near_surface' 'relative_humidity_near_surface'
 'dew_point_temperature' 'humidity_index' 'vertical_temperature_gradient']
class unique entries: ['rd']
expver unique entries: ['xxxx']
stream unique entries: ['iot']
reportype unique entries: [ 1  2 11  3  5]
andate unique entries: [20220616 20220623 20220624 20220630]
antime unique entries: [1800 1400 1500 1600 1200 1300]
groupid@hdr unique entries: [17]
project unique entries: ['I-CHANGE']
date unique entries: [20220616 20220623 20220624 20220630]
time unique entries: [1800 1400 1500 1600 1200 1300]


,platform,source_name,source_id,observation_variable,class,expver,stream,reportype,andate,antime,groupid@hdr,project,date,time,altitude,observed_value,lat,lon,minutes
0,meteotracker,genova_living_lab_1,62ab72c11d8e11061d32002a,air_temperature_near_surface,rd,xxxx,iot,1,20220616,1800,17,I-CHANGE,20220616,1800,149.511093,303.049988,44.400051,8.674493,13.250000
1,meteotracker,genova_living_lab_1,62ab72c11d8e11061d32002a,air_temperature_near_surface,rd,xxxx,iot,1,20220616,1800,17,I-CHANGE,20220616,1800,149.511093,302.250000,44.400135,8.674781,13.300000
2,meteotracker,genova_living_lab_1,62ab72c11d8e11061d32002a,air_temperature_near_surface,rd,xxxx,iot,1,20220616,1800,17,I-CHANGE,20220616,1800,141.148972,301.649994,44.400219,8.675081,13.350000
3,meteotracker,genova_living_lab_1,62ab72c11d8e11061d32002a,air_temperature_near_surface,rd,xxxx,iot,1,20220616,1800,17,I-CHANGE,20220616,1800,141.148972,301.350006,44.400352,8.675219,13.400000
4,meteotracker,genova_living_lab_1,62ab72c11d8e11061d32002a,air_temperature_near_surface,rd,xxxx,iot,1,20220616,1800,17,I-CHANGE,20220616,1800,141.148972,301.049988,44.400509,8.675051,13.450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,meteotracker,bologna_living_lab_4,62bda5d4edb2ca4efd101c4d,humidity_index,rd,xxxx,iot,3,20220630,1300,17,I-CHANGE,20220630,1300,141.148972,319.850006,44.499550,11.354349,12.833333
4794,meteotracker,bologna_living_lab_4,62bda5d4edb2ca4efd101c4d,humidity_index,rd,xxxx,iot,3,20220630,1300,17,I-CHANGE,20220630,1300,141.148972,319.950012,44.499550,11.354349,12.883333
4795,meteotracker,bologna_living_lab_4,62bda5d4edb2ca4efd101c4d,humidity_index,rd,xxxx,iot,3,20220630,1300,17,I-CHANGE,20220630,1300,141.148972,318.750000,44.499561,11.354325,12.933333
4796,meteotracker,bologna_living_lab_4,62bda5d4edb2ca4efd101c4d,humidity_index,rd,xxxx,iot,3,20220630,1300,17,I-CHANGE,20220630,1300,141.148972,319.250000,44.499538,11.354254,12.983334


In [9]:
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326)

geo_df.explore(column = "altitude")

NameError: name 'gpd' is not defined

## Filtering data granules with SQL queries

In [10]:
mars_request = {
    "project" : "public", # This key will be used for access control and will change to 'ichange' in future versions
    "platform" : platform,
    "observation_variable" : parameter,
    "datetime": start_time.isoformat(),
    "filter" : f"select * from result where source_id = '{source_id}';",
    "format" : "csv",
}

print(mars_request)


data = polytope_client.retrieve('iot', mars_request)
print(f"{len(data) = }")

df = pd.read_csv(BytesIO(data[0]))
# assert(list(df["source_id"].unique()) == [source_id,])

df

2024-10-09 13:59:41 - INFO - Sending request...
{'request': "datetime: '2024-04-09T11:31:24.011000+00:00'\n"
            'filter: select * from result where source_id = '
            "'66152710f5c2613269425a01';\n"
            'format: csv\n'
            'observation_variable: air_temperature_near_surface\n'
            'platform: meteotracker\n'
            'project: public\n',
 'verb': 'retrieve'}


{'project': 'public', 'platform': 'meteotracker', 'observation_variable': 'air_temperature_near_surface', 'datetime': '2024-04-09T11:31:24.011000+00:00', 'filter': "select * from result where source_id = '66152710f5c2613269425a01';", 'format': 'csv'}


2024-10-09 13:59:41 - INFO - Request accepted. Please poll ../requests/1cd43cbd-1233-4b5f-b785-7da6b94803cd for status
2024-10-09 13:59:41 - INFO - Checking request status (1cd43cbd-1233-4b5f-b785-7da6b94803cd)...
2024-10-09 13:59:42 - INFO - The current status of the request is 'queued'
2024-10-09 13:59:43 - INFO - The current status of the request is 'processed'
2024-10-09 13:59:43 - INFO - Starting data download (application/octet-stream)...


len(data) = 1


,platform,source_name,source_id,observation_variable,class,expver,stream,reportype,andate,antime,groupid@hdr,project,date,time,altitude,observed_value,lat,lon,minutes


In [11]:
mars_request = {
    "project" : "public", # This key will be used for access control and will change to 'ichange' in future versions
    "platform" : platform,
    "observation_variable" : parameter,
    "datetime": start_time.isoformat(),
    "filter" : f"select lat,lon,observed_value from result where source_id = '{source_id}';",
    "format" : "csv",
}

print(mars_request)


data = polytope_client.retrieve('iot', mars_request)
print(f"{len(data) = }")

df = pd.read_csv(BytesIO(data[0]))
# assert(list(df["source_id"].unique()) == [source_id,])

df

2024-10-09 14:00:24 - INFO - Sending request...
{'request': "datetime: '2024-04-09T11:31:24.011000+00:00'\n"
            'filter: select lat,lon,observed_value from result where source_id '
            "= '66152710f5c2613269425a01';\n"
            'format: csv\n'
            'observation_variable: air_temperature_near_surface\n'
            'platform: meteotracker\n'
            'project: public\n',
 'verb': 'retrieve'}


{'project': 'public', 'platform': 'meteotracker', 'observation_variable': 'air_temperature_near_surface', 'datetime': '2024-04-09T11:31:24.011000+00:00', 'filter': "select lat,lon,observed_value from result where source_id = '66152710f5c2613269425a01';", 'format': 'csv'}


2024-10-09 14:00:25 - INFO - Request accepted. Please poll ../requests/0bbc4273-c88c-4fd3-aca9-c83263c778fb for status
2024-10-09 14:00:25 - INFO - Checking request status (0bbc4273-c88c-4fd3-aca9-c83263c778fb)...
2024-10-09 14:00:25 - INFO - The current status of the request is 'processed'
2024-10-09 14:00:25 - INFO - Starting data download (application/octet-stream)...


len(data) = 1


,lat,lon,observed_value


In [ ]:
mars_request = {
    "project" : "public",
    "platform" : platform,
    "observation_variable" : parameter,
    "datetime": start_time.isoformat(),
    "filter" : f"select * from result where source_name like 'genova_living_lab%';",
    "format" : "csv",
}

data = polytope_client.retrieve('iot', mars_request)
print(f"{len(data) = }")

df = pd.read_csv(BytesIO(data[0]))
df

## Direct REST API


In [ ]:
mars_request = {
    "project" : "public", # This key will be used for access control and will change to 'ichange' in future versions
    "platform" : platform,
    "observation_variable" : 'air_temperature_near_surface',
    "datetime": start_time.isoformat(),
    "filter" : f"select * from result where source_id = '{source_id}';",
    "format" : "csv",
}

list_resp = session.get(url + "list", params = mars_request)
list_resp.json()

In [ ]:
data = session.get(url + "retrieve", params = mars_request)

In [ ]:
data.content